
# SE-Residual Block (Squeeze-and-Excitation + Residual) — Baştan Sona

Bu notebook yalnızca **SE-Residual** (SE attention + residual bağlantı) yapısını anlatır:  
- SE’nin ne yaptığı  
- Residual bloğun neresine yerleştiği  
- Pre-Act / Post-Act yerleşimleri  
- Bir modele **temiz entegrasyon**  
- Hızlı kontrol listesi ve shape testleri  

> Notasyon  
- `F(x)`: Residual yolun dönüşümü (conv/BN/act + SE)  
- `skip(x)`: identity veya 1×1 projeksiyon  
- Çıkış: `y = F(x) + skip(x)`



## 1) SE nedir? (Channel attention)

SE (Squeeze-and-Excitation) şunu yapar:

1. **Squeeze**: Feature map’i kanal başına tek sayıya indirir (Global Average Pooling).  
2. **Excitation**: Küçük bir MLP ile her kanal için önem katsayısı üretir.  
3. **Sigmoid**: Katsayıları `[0,1]` aralığına sıkıştırır.  
4. **Scale**: Feature map’i kanal bazında çarpar: `x * w`.

Kritik nokta:  
- SE **yeni uzamsal bilgi üretmez**.  
- Var olan kanalları **yeniden ağırlıklandırır**.


In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F

def count_params(m: nn.Module) -> int:
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

def demo(m: nn.Module, x: torch.Tensor, name: str):
    with torch.no_grad():
        y = m(x)
    print(f"{name:>16} | in {tuple(x.shape)} -> out {tuple(y.shape)} | params {count_params(m):,}")



## 2) SE modülü (tek başına)

SE modülü tek başına residual değildir.  
SE, **residual bloğun içindeki** `F(x)` dönüşümüne takılan bir ektir.


In [2]:

class SEBlock(nn.Module):
    def __init__(self, channels: int, reduction: int = 16, min_hidden: int = 4):
        super().__init__()
        hidden = max(min_hidden, channels // reduction)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Conv2d(channels, hidden, 1, bias=True),
            nn.SiLU(inplace=True),
            nn.Conv2d(hidden, channels, 1, bias=True),
            nn.Sigmoid(),
        )

    def forward(self, x):
        w = self.fc(self.pool(x))   # (B,C,1,1)
        return x * w                # kanal bazlı ölçekleme

x = torch.randn(2, 64, 20, 20)
demo(SEBlock(64), x, "SEBlock")


         SEBlock | in (2, 64, 20, 20) -> out (2, 64, 20, 20) | params 580



## 3) SE-Residual: SE nereye konur?

Pratikte en yaygın (ve en “standart”) yerleşim şudur:

- `Conv/BN/Act` ile feature çıkar  
- ikinci conv ile dönüşümü bitir  
- **SE’yi en sona** (toplamadan önce) uygula  
- sonra `+ skip`

Şema:

```bash
x ───────────────┐
                 ├── (+) ──> çıktı
Conv → ... → Conv → SE
```

Bu yüzden SE çoğu zaman “residual bloğun sonuna eklenen channel attention” gibi görünür.



## 4) Dikkat edilmesi gereken 4 nokta (SE-Residual Checklist)

SE-Residual eklerken hatanın %90’ı şu 4 yerden çıkar:

1. **Toplama öncesi shape uyumu**  
   - `F(x)` ve `skip(x)` aynı `(C,H,W)` olmalı.

2. **Stride/downsample senaryosu**  
   - Stride=2 ise skip yolu da downsample edilmeli (1×1 conv + stride).

3. **Kanal geçişi**  
   - `cin != cout` ise skip yolu projeksiyon ister.

4. **SE’nin konumu**  
   - SE, genelde `F(x)` dönüşümünün **en sonuna** (toplamadan hemen önce) konur.



## 5) Post-Act SE-Residual Block (klasik yerleşim)

Bu versiyonda residual yol “post-activation” kalıbına yakındır:

- `Conv → BN → Act → Conv → BN → SE → +skip → Act`

Not: En sondaki aktivasyon tasarıma göre opsiyonel olabilir. Burada pratikte sık görülen şekilde ekliyoruz.


In [3]:

class ConvBNAct(nn.Module):
    def __init__(self, cin, cout, k=3, s=1, p=1, act="silu"):
        super().__init__()
        self.conv = nn.Conv2d(cin, cout, k, stride=s, padding=p, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.SiLU(inplace=True) if act == "silu" else nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

class SEResBlock_PostAct(nn.Module):
    def __init__(self, cin, cout, stride=1, reduction=16):
        super().__init__()
        self.conv1 = ConvBNAct(cin, cout, k=3, s=stride, p=1, act="silu")
        self.conv2 = nn.Sequential(
            nn.Conv2d(cout, cout, 3, padding=1, bias=False),
            nn.BatchNorm2d(cout),
        )
        self.se = SEBlock(cout, reduction=reduction)

        self.proj = None
        if stride != 1 or cin != cout:
            self.proj = nn.Sequential(
                nn.Conv2d(cin, cout, 1, stride=stride, bias=False),
                nn.BatchNorm2d(cout),
            )
        self.out_act = nn.SiLU(inplace=True)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)
        y = self.conv1(x)
        y = self.conv2(y)
        y = self.se(y)
        y = y + skip
        return self.out_act(y)

demo(SEResBlock_PostAct(32, 32), torch.randn(2,32,32,32), "SEPostAct s1")
demo(SEResBlock_PostAct(32, 64, stride=2), torch.randn(2,32,32,32), "SEPostAct s2")


    SEPostAct s1 | in (2, 32, 32, 32) -> out (2, 32, 32, 32) | params 18,852
    SEPostAct s2 | in (2, 32, 32, 32) -> out (2, 64, 16, 16) | params 58,308



## 6) Pre-Act SE-Residual Block (ResNet v2 çizgisi)

Bu versiyonda BN+Act, conv’dan önce gelir:

- `BN → Act → Conv → BN → Act → Conv → SE → +skip`

Bu stil, çok derin yapılarda daha stabil optimizasyon sağlayabilir.


In [4]:

class SEResBlock_PreAct(nn.Module):
    def __init__(self, cin, cout, stride=1, reduction=16):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(cin)
        self.conv1 = nn.Conv2d(cin, cout, 3, stride=stride, padding=1, bias=False)

        self.bn2 = nn.BatchNorm2d(cout)
        self.conv2 = nn.Conv2d(cout, cout, 3, padding=1, bias=False)

        self.se = SEBlock(cout, reduction=reduction)

        self.proj = None
        if stride != 1 or cin != cout:
            self.proj = nn.Conv2d(cin, cout, 1, stride=stride, bias=False)

    def forward(self, x):
        skip = x if self.proj is None else self.proj(x)

        y = F.silu(self.bn1(x), inplace=True)
        y = self.conv1(y)

        y = F.silu(self.bn2(y), inplace=True)
        y = self.conv2(y)

        y = self.se(y)
        return y + skip

demo(SEResBlock_PreAct(32, 32), torch.randn(2,32,32,32), "SEPreAct s1")
demo(SEResBlock_PreAct(32, 64, stride=2), torch.randn(2,32,32,32), "SEPreAct s2")


     SEPreAct s1 | in (2, 32, 32, 32) -> out (2, 32, 32, 32) | params 18,852
     SEPreAct s2 | in (2, 32, 32, 32) -> out (2, 64, 16, 16) | params 58,116



## 7) SE-Residual bir modele nasıl entegre edilir? (basitten ileriye)

En kolay yol: Modeli “stage” mantığıyla kurmak.

- `stem`: giriş conv
- `stage`: aynı boyutta birden fazla SE-Residual blok
- `stride=2`: bir stage’den diğerine geçerken çözünürlük düşürme (downsample)

Aşağıdaki örnek küçük bir SE-ResNet iskeletidir (CIFAR benzeri boyutlarla).


In [5]:

def make_stage(block_cls, cin, cout, n_blocks, first_stride, reduction):
    layers = [block_cls(cin, cout, stride=first_stride, reduction=reduction)]
    for _ in range(1, n_blocks):
        layers.append(block_cls(cout, cout, stride=1, reduction=reduction))
    return nn.Sequential(*layers)

class SEResNetTiny(nn.Module):
    def __init__(self, in_ch=3, num_classes=10, base=32, n=2, reduction=16, preact=False):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(in_ch, base, 3, padding=1, bias=False),
            nn.BatchNorm2d(base),
            nn.SiLU(inplace=True),
        )

        block = SEResBlock_PreAct if preact else SEResBlock_PostAct
        ch1, ch2, ch3 = base, base*2, base*4

        self.stage1 = make_stage(block, ch1, ch1, n_blocks=n, first_stride=1, reduction=reduction)
        self.stage2 = make_stage(block, ch1, ch2, n_blocks=n, first_stride=2, reduction=reduction)
        self.stage3 = make_stage(block, ch2, ch3, n_blocks=n, first_stride=2, reduction=reduction)

        self.head = nn.Sequential(
            nn.BatchNorm2d(ch3),
            nn.SiLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
        )
        self.fc = nn.Linear(ch3, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.head(x).flatten(1)
        return self.fc(x)

m1 = SEResNetTiny(n=2, preact=False)
m2 = SEResNetTiny(n=2, preact=True)
demo(m1, torch.randn(2,3,32,32), "SEResNetTiny post")
demo(m2, torch.randn(2,3,32,32), "SEResNetTiny pre")


SEResNetTiny post | in (2, 3, 32, 32) -> out (2, 10) | params 702,986
SEResNetTiny pre | in (2, 3, 32, 32) -> out (2, 10) | params 702,410



## 8) Hızlı doğrulama: shape + grad

SE-Residual entegrasyonunda “çalışıyor mu?” kontrolü için 2 test yeter:

- **Shape testi**: forward patlıyor mu, çıktı boyutu doğru mu?
- **Grad testi**: `.backward()` akıyor mu?

Bu hücre, tek satırda kontrol eder.


In [6]:

def quick_check_model(m: nn.Module, x: torch.Tensor):
    m.train()
    y = m(x)
    loss = y.mean()
    loss.backward()
    print("ok | out:", tuple(y.shape))

x = torch.randn(2, 3, 32, 32, requires_grad=True)
quick_check_model(SEResNetTiny(n=2, preact=False), x)


ok | out: (2, 10)



## 9) Pratik notlar (ne zaman işe yarar?)

SE-Residual çoğunlukla şuralarda faydalı olur:
- Kanal sayısı arttıkça (orta/derin stage’ler)  
- Özellikle “hangi feature daha önemli?” seçiminin kritik olduğu işlerde

Dikkat:
- SE, parametre ve gecikme ekler.  
- Çok hafif modellerde (edge) bazen ECA gibi daha ucuz alternatifler tercih edilir.



## 10) En kısa özet

- Residual: `y = F(x) + skip(x)` (öğrenmeyi stabil yapar)  
- SE: kanalları ağırlıklandırır: `x * w`  
- SE-Residual: SE’yi `F(x)` içine koyarsın (toplamadan hemen önce)

En kritik entegrasyon kuralı:
> Toplama yapılacaksa `F(x)` ve `skip(x)` aynı shape’e getirilir.
